**[Feature Engineering Home Page](https://www.kaggle.com/learn/feature-engineering)**

---


# Introduction

In this exercise you'll apply more advanced encodings to encode the categorical variables ito improve your classifier model. The encodings you will implement are:

- Count Encoding
- Target Encoding
- CatBoost Encoding

You'll refit the classifier after each encoding to check its performance on hold-out data. 

Begin by running the next code cell to set up the notebook.

In [1]:
# Set up code checking
# This can take a few seconds
from learntools.core import binder
binder.bind(globals())
from learntools.feature_engineering.ex2 import *

/opt/conda/lib/python3.7/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/opt/conda/lib/python3.7/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


The next code cell repeats the work that you did in the previous exercise.

In [2]:
import numpy as np
import pandas as pd
from sklearn import preprocessing, metrics
import lightgbm as lgb

clicks = pd.read_parquet('../input/feature-engineering-data/baseline_data.pqt')

In [3]:
clicks.head()

,ip,app,device,os,channel,click_time,attributed_time,is_attributed,day,hour,minute,second
0,27226,3,1,13,120,2017-11-06 15:13:23,None,0,6,15,13,23
1,110007,35,1,13,10,2017-11-06 15:41:07,2017-11-07 08:17:19,1,6,15,41,7
2,1047,6,1,13,157,2017-11-06 15:42:32,None,0,6,15,42,32
3,76270,3,1,13,120,2017-11-06 15:56:17,None,0,6,15,56,17
4,57862,3,1,13,120,2017-11-06 15:57:01,None,0,6,15,57,1


Next, we define a couple functions that you'll use to test the encodings that you implement in this exercise.

In [4]:
def get_data_splits(dataframe, valid_fraction=0.1):
    """Splits a dataframe into train, validation, and test sets.

    First, orders by the column 'click_time'. Set the size of the 
    validation and test sets with the valid_fraction keyword argument.
    """

    dataframe = dataframe.sort_values('click_time')
    valid_rows = int(len(dataframe) * valid_fraction)
    train = dataframe[:-valid_rows * 2]
    # valid size == test size, last two sections of the data
    valid = dataframe[-valid_rows * 2:-valid_rows]
    test = dataframe[-valid_rows:]
    
    return train, valid, test

def train_model(train, valid, test=None, feature_cols=None):
    if feature_cols is None:
        feature_cols = train.columns.drop(['click_time', 'attributed_time',
                                           'is_attributed'])
    dtrain = lgb.Dataset(train[feature_cols], label=train['is_attributed'])
    dvalid = lgb.Dataset(valid[feature_cols], label=valid['is_attributed'])
    
    param = {'num_leaves': 64, 'objective': 'binary', 
             'metric': 'auc', 'seed': 7}
    num_round = 1000
    bst = lgb.train(param, dtrain, num_round, valid_sets=[dvalid], 
                    early_stopping_rounds=20, verbose_eval=False)
    
    valid_pred = bst.predict(valid[feature_cols])
    valid_score = metrics.roc_auc_score(valid['is_attributed'], valid_pred)
    print(f"Validation AUC score: {valid_score}")
    
    if test is not None: 
        test_pred = bst.predict(test[feature_cols])
        test_score = metrics.roc_auc_score(test['is_attributed'], test_pred)
        return bst, valid_score, test_score
    else:
        return bst, valid_score

Run this cell to get a baseline score. 

In [5]:
print("Baseline model")
train, valid, test = get_data_splits(clicks)
_ = train_model(train, valid)

Baseline model
Validation AUC score: 0.9622743228943659


### 1) Categorical encodings and leakage

These encodings are all based on statistics calculated from the dataset like counts and means. 

Considering this, what data should you be using to calculate the encodings?  Specifically, can you use the validation data?  Can you use the test data?

Run the following line after you've decided your answer.

In [6]:
# Check your answer (Run this code cell to receive credit!)
q_1.solution()

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> You should calculate the encodings from the training set only. If you include data from the validation and test sets into the encodings, you'll overestimate the model's performance. You should in general be vigilant to avoid leakage, that is, including any information from the validation and test sets into the model. For a review on this topic, see our lesson on [data leakage](https://www.kaggle.com/alexisbcook/data-leakage)

### 2) Count encodings

Begin by running the next code cell to get started.

In [7]:
import category_encoders as ce

cat_features = ['ip', 'app', 'device', 'os', 'channel']
train, valid, test = get_data_splits(clicks)

Next, encode the categorical features `['ip', 'app', 'device', 'os', 'channel']` using the count of each value in the data set. 
- Using `CountEncoder` from the `category_encoders` library, fit the encoding using the categorical feature columns defined in `cat_features`. 
- Then apply the encodings to the train and validation sets, adding them as new columns with names suffixed `"_count"`.

In [35]:
# Create the count encoder
count_enc = ce.CountEncoder(cols = cat_features)

# Learn encoding from the training set
count_enc.fit(train[cat_features])

# Apply encoding to the train and validation sets as new columns
# Make sure to add `_count` as a suffix to the new columns

# DataFrame.add_suffix(suffix)
# string.join(iterable)
# transform(y)
train_encoded = train.join(count_enc.transform(train[cat_features]).add_suffix("_count"))
valid_encoded = valid.join(count_enc.transform(valid[cat_features]).add_suffix("_count"))

# Check your answer
q_2.check()
train_encoded[cat_features]

/opt/conda/lib/python3.7/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


<IPython.core.display.Javascript object>

<span style="color:#cc3333">Incorrect:</span> Expected `train_encoded` to have column `ip_count`

,app,device,os,channel
0,3,1,13,120
1,35,1,13,10
2,6,1,13,157
3,3,1,13,120
4,3,1,13,120
...,...,...,...,...
1840445,18,1,17,26
1840449,15,1,19,75
1840446,48,1,19,103
1840443,19,6,21,59


In [16]:
train_encoded

,ip,app,device,os,channel,click_time,attributed_time,is_attributed,day,hour,minute,second,ip_count,app_count,device_count,os_count,channel_count
0,27226,3,1,13,120,2017-11-06 15:13:23,None,0,6,15,13,23,68,292254,1648091,370652,26760
1,110007,35,1,13,10,2017-11-06 15:41:07,2017-11-07 08:17:19,1,6,15,41,7,4,60114,1648091,370652,41256
2,1047,6,1,13,157,2017-11-06 15:42:32,None,0,6,15,42,32,118,19564,1648091,370652,31221
3,76270,3,1,13,120,2017-11-06 15:56:17,None,0,6,15,56,17,29,292254,1648091,370652,26760
4,57862,3,1,13,120,2017-11-06 15:57:01,None,0,6,15,57,1,31,292254,1648091,370652,26760
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1840445,11718,18,1,17,26,2017-11-09 04:50:14,None,0,9,4,50,14,28,129507,1648091,87078,39586
1840449,32849,15,1,19,75,2017-11-09 04:50:14,None,0,9,4,50,14,51,140369,1648091,420998,84548
1840446,249422,48,1,19,103,2017-11-09 04:50:14,2017-11-09 04:53:15,1,9,4,50,14,1,845,1648091,420998,7403
1840443,232256,19,6,21,59,2017-11-09 04:50:14,2017-11-09 04:50:59,1,9,4,50,14,1,104701,492,15303,107747


In [12]:
# Uncomment if you need some guidance
# q_2.hint()
# q_2.solution()

Run the next code cell to see how count encoding changes the results.

In [17]:
# Train the model on the encoded datasets
# This can take around 30 seconds to complete
_ = train_model(train_encoded, valid_encoded)

Validation AUC score: 0.9653051135205329


Count encoding improved our model's score!

### 3) Why is count encoding effective?
At first glance, it could be surprising that count encoding helps make accurate models. 
Why do you think is count encoding is a good idea, or how does it improve the model score?

Run the following line after you've decided your answer.

In [18]:
# Check your answer (Run this code cell to receive credit!)
q_3.solution()

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
    Rare values tend to have similar counts (with values like 1 or 2), so you can classify rare 
    values together at prediction time. Common values with large counts are unlikely to have 
    the same exact count as other values. So, the common/important values get their own 
    grouping.
    

### 4) Target encoding

Here you'll try some supervised encodings that use the labels (the targets) to transform categorical features. The first one is target encoding. 
- Create the target encoder from the `category_encoders` library. 
- Then, learn the encodings from the training dataset, apply the encodings to all the datasets, and retrain the model.

In [25]:
# Create the target encoder. You can find this easily by using tab completion.
# Start typing ce. the press Tab to bring up a list of classes and functions.
target_enc = ce.TargetEncoder(cols = cat_features)

# Learn encoding from the training set. Use the 'is_attributed' column as the target.
# fit(y)
target_enc.fit(train[cat_features], train['is_attributed'])

# Apply encoding to the train and validation sets as new columns
# Make sure to add `_target` as a suffix to the new columns

# DataFrame.add_suffix(suffix)
# string.join(iterable)
# transform(y)
train_encoded = train.join(target_enc.transform(train[cat_features]).add_suffix('_target'))
valid_encoded = valid.join(target_enc.transform(valid[cat_features]).add_suffix('_target'))

# Check your answer
q_4.check()
train_encoded

/opt/conda/lib/python3.7/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

,ip,app,device,os,channel,click_time,attributed_time,is_attributed,day,hour,minute,second,ip_target,app_target,device_target,os_target,channel_target
0,27226,3,1,13,120,2017-11-06 15:13:23,None,0,6,15,13,23,0.073529,0.028328,0.152087,0.138712,0.034043
1,110007,35,1,13,10,2017-11-06 15:41:07,2017-11-07 08:17:19,1,6,15,41,7,0.247523,0.995841,0.152087,0.138712,0.950262
2,1047,6,1,13,157,2017-11-06 15:42:32,None,0,6,15,42,32,0.135593,0.009252,0.152087,0.138712,0.019378
3,76270,3,1,13,120,2017-11-06 15:56:17,None,0,6,15,56,17,0.103448,0.028328,0.152087,0.138712,0.034043
4,57862,3,1,13,120,2017-11-06 15:57:01,None,0,6,15,57,1,0.096774,0.028328,0.152087,0.138712,0.034043
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1840445,11718,18,1,17,26,2017-11-09 04:50:14,None,0,9,4,50,14,0.071429,0.048499,0.152087,0.109913,0.043930
1840449,32849,15,1,19,75,2017-11-09 04:50:14,None,0,9,4,50,14,0.098039,0.021686,0.152087,0.157243,0.009202
1840446,249422,48,1,19,103,2017-11-09 04:50:14,2017-11-09 04:53:15,1,9,4,50,14,0.197764,0.951479,0.152087,0.157243,0.970012
1840443,232256,19,6,21,59,2017-11-09 04:50:14,2017-11-09 04:50:59,1,9,4,50,14,0.197764,0.948415,0.953252,0.953408,0.953326


In [26]:
# Uncomment these if you need some guidance
#q_4.hint()
# q_4.solution()

Run the next cell to see how target encoding affects your results.

In [27]:
_ = train_model(train_encoded, valid_encoded)

Validation AUC score: 0.9540530347873288


### 5) Try removing IP encoding

If you leave `ip` out of the encoded features and retrain the model with target encoding, you should find that the score increases and is above the baseline score! Why do you think the score is below baseline when we encode the IP address but above baseline when we don't?

Run the following line after you've decided your answer.

In [28]:
# Check your answer (Run this code cell to receive credit!)
q_5.solution()

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
    Target encoding attempts to measure the population mean of the target for each 
    level in a categorical feature. This means when there is less data per level, the 
    estimated mean will be further away from the "true" mean, there will be more variance. 
    There is little data per IP address so it's likely that the estimates are much noisier
    than for the other features. The model will rely heavily on this feature since it is 
    extremely predictive. This causes it to make fewer splits on other features, and those
    features are fit on just the errors left over accounting for IP address. So, the 
    model will perform very poorly when seeing new IP addresses that weren't in the 
    training data (which is likely most new data). Going forward, we'll leave out the IP feature when trying
    different encodings.
    

### 6) CatBoost Encoding

The CatBoost encoder is supposed to work well with the LightGBM model. Encode the categorical features with `CatBoostEncoder` and train the model on the encoded data again.

In [30]:
# Remove IP from the encoded features
cat_features = ['app', 'device', 'os', 'channel']

# Create the CatBoost encoder
cb_enc = ce.CatBoostEncoder(cols=cat_features, random_state=7)

# Learn encoding from the training set
# fit(y)
cb_enc.fit(train[cat_features], train['is_attributed'])

# Apply encoding to the train and validation sets as new columns
# Make sure to add `_cb` as a suffix to the new columns

# DataFrame.add_suffix(suffix)
# string.join(iterable)
# transform(y)
train_encoded = train.join(cb_enc.transform(train[cat_features]).add_suffix('_cb'))
valid_encoded = valid.join(cb_enc.transform(valid[cat_features]).add_suffix('_cb'))

# Check your answer
q_6.check()

/opt/conda/lib/python3.7/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [32]:
# Uncomment these if you need some guidance
#q_6.hint()
# q_6.solution()

Run the next code cell to see how the CatBoost encoder changes your results.

In [33]:
_ = train_model(train_encoded, valid_encoded)

Validation AUC score: 0.962868024575231


# Keep Going

Now you are ready to **[generate completely new features](https://www.kaggle.com/matleonard/feature-generation)** from the data.

---
**[Feature Engineering Home Page](https://www.kaggle.com/learn/feature-engineering)**





*Have questions or comments? Visit the [Learn Discussion forum](https://www.kaggle.com/learn-forum/161443) to chat with other Learners.*